## Understanding RNNs from first principles

> Notes on the blog post by Terence Parr: https://explained.ai/rnn/index.html

---

## The goal: meaningful vectors representing words

To understand what's going on inside an RNN, let's re-invent the order-sensitive encodings generated by an RNN using a simple classification problem. Imagine that we have three words for cat in three different languages. Given a word, we'd like to classify it as English, French, or German:

![](https://i.imgur.com/QiYhC19.png)

In order to numericalize data before training a model, we can encode the targets as classes 0, 1, and 2, which works great. Unfortunately, we can't convert the cat words to unique integers because we get nonsense like 0->0, as shown on the right above.

Instead of a single number per word, we need to come up with a vector of numbers to represent each word. We don't need to know what the vector elements are per se, just that they somehow meaningfully represent a word in some high-dimensional space:

![](https://explained.ai/rnn/images/vector-to-num.svg)

As an analogy, consider the common tactic of breaking apart a single date feature (often represented as the number of seconds since 1970) into a vector of its constituent components like hour, minute, day, month, year.

Once we have these meaningful feature vectors (for cat, chat, and katze), we can use them to train a random forest or any other classifier. So this article is all about how we find suitable vectors. To do that, let's baby step through some possible approaches to arrive at the RNN solution.





## Encoding words as integers

The first thing we have to do is split apart the words into a sequence of characters and encode those characters using a character vocabulary. Computing the vocabulary is straightforward. Any unique integers will work for the vocabulary, but the implementation is simpler if we use consecutive integers starting from zero:

![](https://i.imgur.com/uhnu92r.png)

In [1]:
vocab = {c:i for i,c in enumerate("acehktz")}
print (vocab)

{'a': 0, 'c': 1, 'e': 2, 'h': 3, 'k': 4, 't': 5, 'z': 6}


Unfortunately, the feature vectors for different translations of the word "cat" have different lengths (three, four, and five). A simple way to convert these variable length character sequences into a single feature is to add up the vocabulary character encodings, which we can do with a trivial loop:



In [2]:
for word in ["cat", "chat", "katze"]:
    x = [vocab[letter] for letter in word]
    h = 0
    for t in range(len(x)):
        h+=x[t]

    print (f'{word} -> {h}')

cat -> 6
chat -> 9
katze -> 17


This is not a great solution: First, because it's unclear that 6, 9, and 17 meaningfully distinguish between the three translations of cat. More importantly, though, this encoding is order independent. For example, a-c-t has the same encoding as c-a-t:

A simple way to make the encoding order dependent is to multiply previous h values by, say, 2. This performs a scale and add operation very much like what we'd find in a hash function. Scalar 2 is not some magic number—I just chose it randomly, but it's a value we could learn if we created and optimized a loss function.





In [4]:
for word in ["cat", "act"]:
    x = [vocab[letter] for letter in word]
    h = 0
    print (x)
    for t in range(len(x)):
        print (f'h_t-1: {h}, x_t: {x[t]}')
        h = 2*h + x[t]
        print (f'h_t = 2.h_t-1 + x_t = {h}')
    
    print (f'{word} -> {h}')

[1, 0, 5]
h_t-1: 0, x_t: 1
h_t = 2.h_t-1 + x_t = 1
h_t-1: 1, x_t: 0
h_t = 2.h_t-1 + x_t = 2
h_t-1: 2, x_t: 5
h_t = 2.h_t-1 + x_t = 9
cat -> 9
[0, 1, 5]
h_t-1: 0, x_t: 0
h_t = 2.h_t-1 + x_t = 0
h_t-1: 0, x_t: 1
h_t = 2.h_t-1 + x_t = 1
h_t-1: 1, x_t: 5
h_t = 2.h_t-1 + x_t = 7
act -> 7


That inner loop is equivalent to the following recurrence relation:

![](https://explained.ai/rnn/images/blkeqn-3048294F838BEC17B89CE0457E8359E8.svg)

The recurrence just says that the value of h at iteration t is twice its previous value plus the encoding of the t_th  character; t moves from 1 to m for m characters in the word. For x = cat we get three values of h beyond our initial value:

![](https://explained.ai/rnn/images/blkeqn-33E117DC61B77D31604D90FAB16799E3.svg)


<img src = "https://i.imgur.com/njh0cOD.jpeg" width="700" height="600"/>

The key take away here is that, despite having a constant multiplier of 2, each character encoding is multiplied by a different number, depending on its position in the sequence: x_t is multiplied by 2^(m-t) where m is the no of letters. This is why cat and act get different encodings. We've solved the order dependency issue, but it's unlikely that a single integer will ever contain enough information to meaningfully represent a natural language word.





### Aggregating character vectors to encode words

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=306a1d85-aa04-4db6-b565-92149392d58c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>